In [2]:
import pandas as pd
import numpy as np
import os
from PIL import Image
from io import BytesIO
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from keras.regularizers import l2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix

In [3]:
def create_image_df(folder_path, is_positive, target_size=(256, 256)):
    def resize_with_padding(img, target_size):
        # Resize the image while keeping the aspect ratio
        img.thumbnail(target_size)
        # Create a new image with the target size and a white background
        new_img = Image.new("L", target_size, color=255)  # color for white background
        # Paste the resized image onto the new image, centered
        new_img.paste(img, ((target_size[0] - img.size[0]) // 2, (target_size[1] - img.size[1]) // 2))
        img_arr = np.array(new_img) / 255.0  # Normalize pixel values
        # img_arr = np.expand_dims(img_arr, axis=0)  # Add batch dimension
        img_arr = np.expand_dims(img_arr, axis=-1)  # Add channel dimension for grayscale
        return new_img, img_arr
    
    # List to store file names and images
    file_data = []
    
    # Loop through all files in the folder
    for file in os.listdir(folder_path):
        # Check if the file is an image file
        if file.endswith(('.jpg', '.jpeg', '.png', '.gif')):
            # Open the image file
            image_path = os.path.join(folder_path, file)
            if is_positive:
                y_val = 1
            else:
                y_val = 0
            with Image.open(image_path) as image:
                # Convert to grayscale
                if image.mode != 'L':
                    grayscale_image = image.convert('L')
                image, image_arr = resize_with_padding(image, target_size)
                # Save the image to a byte array
                img_byte_arr = BytesIO()
                image.save(img_byte_arr, format='JPEG')
                img_byte_arr = img_byte_arr.getvalue()
                # Append the file name and image to the list
                file_data.append({'File_Name': file, 'Image': img_byte_arr, 'Array': image_arr, 'y_val': y_val})
    
    # Create a DataFrame from the list of dictionaries
    return pd.DataFrame(file_data)



In [4]:
normal_test_df = create_image_df('./test/NORMAL', False)
pneumonia_test_df = create_image_df('./test/PNEUMONIA', True)
normal_train_df = create_image_df('./train/NORMAL', False)
pneumonia_train_df = create_image_df('./train/PNEUMONIA', True)
normal_val_df = create_image_df('./val/NORMAL', False)
pneumonia_val_df = create_image_df('./val/PNEUMONIA', True)

normal_train_df

,File_Name,Image,Array,y_val
0,IM-0115-0001.jpeg,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,"[[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1...",0
1,IM-0117-0001.jpeg,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,"[[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1...",0
2,IM-0119-0001.jpeg,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,"[[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1...",0
3,IM-0122-0001.jpeg,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,"[[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1...",0
4,IM-0125-0001.jpeg,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,"[[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1...",0
...,...,...,...,...
1336,NORMAL2-IM-1406-0001.jpeg,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,"[[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1...",0
1337,NORMAL2-IM-1412-0001.jpeg,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,"[[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1...",0
1338,NORMAL2-IM-1419-0001.jpeg,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,"[[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1...",0
1339,NORMAL2-IM-1422-0001.jpeg,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,"[[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1...",0


In [5]:
pneumonia_train_df

,File_Name,Image,Array,y_val
0,person1000_bacteria_2931.jpeg,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,"[[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1...",1
1,person1000_virus_1681.jpeg,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,"[[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1...",1
2,person1001_bacteria_2932.jpeg,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,"[[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1...",1
3,person1002_bacteria_2933.jpeg,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,"[[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1...",1
4,person1003_bacteria_2934.jpeg,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,"[[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1...",1
...,...,...,...,...
3869,person99_virus_183.jpeg,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,"[[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1...",1
3870,person9_bacteria_38.jpeg,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,"[[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1...",1
3871,person9_bacteria_39.jpeg,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,"[[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1...",1
3872,person9_bacteria_40.jpeg,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,"[[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1...",1


In [6]:
pneumonia_val_df['y_val'].value_counts()

y_val
1    9
Name: count, dtype: int64

In [7]:
pneumonia_train_df['y_val'].value_counts()

y_val
1    3874
Name: count, dtype: int64

In [8]:
train_df = pd.concat([normal_train_df, pneumonia_train_df], ignore_index=True)
train_df

,File_Name,Image,Array,y_val
0,IM-0115-0001.jpeg,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,"[[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1...",0
1,IM-0117-0001.jpeg,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,"[[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1...",0
2,IM-0119-0001.jpeg,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,"[[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1...",0
3,IM-0122-0001.jpeg,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,"[[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1...",0
4,IM-0125-0001.jpeg,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,"[[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1...",0
...,...,...,...,...
5210,person99_virus_183.jpeg,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,"[[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1...",1
5211,person9_bacteria_38.jpeg,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,"[[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1...",1
5212,person9_bacteria_39.jpeg,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,"[[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1...",1
5213,person9_bacteria_40.jpeg,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,"[[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1...",1


In [9]:
test_df = pd.concat([normal_test_df, pneumonia_test_df], ignore_index=True)
test_df

,File_Name,Image,Array,y_val
0,IM-0001-0001.jpeg,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,"[[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1...",0
1,IM-0003-0001.jpeg,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,"[[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1...",0
2,IM-0005-0001.jpeg,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,"[[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1...",0
3,IM-0006-0001.jpeg,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,"[[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1...",0
4,IM-0007-0001.jpeg,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,"[[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1...",0
...,...,...,...,...
619,person96_bacteria_465.jpeg,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,"[[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1...",1
620,person96_bacteria_466.jpeg,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,"[[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1...",1
621,person97_bacteria_468.jpeg,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,"[[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1...",1
622,person99_bacteria_473.jpeg,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,"[[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1...",1


In [10]:
val_df = pd.concat([normal_val_df, pneumonia_val_df], ignore_index=True)
val_df

,File_Name,Image,Array,y_val
0,NORMAL2-IM-1427-0001.jpeg,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,"[[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1...",0
1,NORMAL2-IM-1430-0001.jpeg,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,"[[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1...",0
2,NORMAL2-IM-1431-0001.jpeg,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,"[[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1...",0
3,NORMAL2-IM-1436-0001.jpeg,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,"[[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1...",0
4,NORMAL2-IM-1437-0001.jpeg,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,"[[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1...",0
5,NORMAL2-IM-1438-0001.jpeg,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,"[[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1...",0
6,NORMAL2-IM-1440-0001.jpeg,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,"[[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1...",0
7,NORMAL2-IM-1442-0001.jpeg,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,"[[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1...",0
8,person1588_virus_2762.jpeg,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,"[[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1...",1
9,person1946_bacteria_4874.jpeg,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,"[[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1...",1


In [11]:
# Extract features and labels
X_train = np.stack(train_df['Array'].values)
y_train = train_df['y_val'].values
X_test = np.stack(test_df['Array'].values)
y_test = test_df['y_val'].values
X_val = np.stack(val_df['Array'].values)
y_val = val_df['y_val'].values

In [12]:
y_train

array([0, 0, 0, ..., 1, 1, 1], dtype=int64)

In [13]:
X_train

array([[[[1.],
         [1.],
         [1.],
         ...,
         [1.],
         [1.],
         [1.]],

        [[1.],
         [1.],
         [1.],
         ...,
         [1.],
         [1.],
         [1.]],

        [[1.],
         [1.],
         [1.],
         ...,
         [1.],
         [1.],
         [1.]],

        ...,

        [[1.],
         [1.],
         [1.],
         ...,
         [1.],
         [1.],
         [1.]],

        [[1.],
         [1.],
         [1.],
         ...,
         [1.],
         [1.],
         [1.]],

        [[1.],
         [1.],
         [1.],
         ...,
         [1.],
         [1.],
         [1.]]],


       [[[1.],
         [1.],
         [1.],
         ...,
         [1.],
         [1.],
         [1.]],

        [[1.],
         [1.],
         [1.],
         ...,
         [1.],
         [1.],
         [1.]],

        [[1.],
         [1.],
         [1.],
         ...,
         [1.],
         [1.],
         [1.]],

        ...,

        [[1.],
 

In [14]:
# # Encode labels
# encoder = LabelEncoder()
# y_train_encoded = encoder.fit_transform(y_train)
# y_test_encoded = encoder.transform(y_test)
# y_val_encoded = encoder.transform(y_val)

# # Convert labels to one-hot encoding
# y_train_one_hot = to_categorical(y_train_encoded)
# y_test_one_hot = to_categorical(y_test_encoded)
# y_val_one_hot = to_categorical(y_val_encoded)

In [15]:
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

datagen.fit(X_train)

In [16]:
batch_size = 32
train_generator = datagen.flow(X_train, y_train, batch_size=batch_size)

In [17]:
# # Build the CNN model
# # model = Sequential([
# #     Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 1), kernel_regularizer=l2(0.01)),
# #     MaxPooling2D((2, 2)),
# #     Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(0.01)),
# #     MaxPooling2D((2, 2)),
# #     Conv2D(128, (3, 3), activation='relu', kernel_regularizer=l2(0.01)),
# #     MaxPooling2D((2, 2)),
# #     Flatten(),
# #     Dense(256, activation='relu', kernel_regularizer=l2(0.01)),
# #     Dropout(0.5),
# #     Dense(1, activation='sigmoid', kernel_regularizer=l2(0.01))  # 2 classes: pneumonia, normal
# # ])

# # model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# model = Sequential([
#     Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 1), kernel_regularizer=l2(0.01)),
#     MaxPooling2D((2, 2)),
#     Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(0.01)),
#     MaxPooling2D((2, 2)),
#     Dropout(0.25),
#     Conv2D(128, (3, 3), activation='relu', kernel_regularizer=l2(0.01)),
#     MaxPooling2D((2, 2)),
#     Dropout(0.25),
#     Flatten(),
#     Dense(256, activation='relu', kernel_regularizer=l2(0.01)),
#     Dropout(0.5),
#     Dense(1, activation='sigmoid', kernel_regularizer=l2(0.01))  # 2 classes: pneumonia, normal
# ])

# # Compile the model
# model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# model = Sequential([
#     Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 1), kernel_regularizer=l2(0.005)),
#     MaxPooling2D((2, 2)),
#     Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(0.005)),
#     MaxPooling2D((2, 2)),
#     Conv2D(128, (3, 3), activation='relu', kernel_regularizer=l2(0.005)),
#     MaxPooling2D((2, 2)),
#     Flatten(),
#     Dense(256, activation='relu', kernel_regularizer=l2(0.005)),
#     Dropout(0.5),
#     Dense(1, activation='sigmoid', kernel_regularizer=l2(0.005))  # 2 classes: pneumonia, normal
# ])

# # Compile the model
# model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# model = Sequential([
#     Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 1), kernel_regularizer=l2(0.01)),
#     MaxPooling2D((2, 2)),
#     Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(0.01)),
#     MaxPooling2D((2, 2)),
#     Conv2D(128, (3, 3), activation='relu', kernel_regularizer=l2(0.01)),
#     MaxPooling2D((2, 2)),
#     Flatten(),
#     Dense(256, activation='relu', kernel_regularizer=l2(0.01)),
#     Dropout(0.5),
#     Dense(1, activation='sigmoid', kernel_regularizer=l2(0.01))  # 2 classes: pneumonia, normal
# ])

# # Compile the model
# model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

In [18]:
# # Define early stopping
# early_stopping = EarlyStopping(monitor='val_loss', patience=3)

In [19]:
# y_train_one_hot[0]

In [20]:
# y_val_one_hot[0]

In [21]:
# # Train the model
# history = model.fit(train_generator, 
#                     validation_data=(X_val, y_val), 
#                     epochs=20, 
#                     batch_size=batch_size, 
#                     )

In [22]:
# # with dropout .25
# # Evaluate the model
# test_loss, test_acc = model.evaluate(X_test, y_test)
# print(f'Test accuracy: {test_acc}')

In [23]:
# Build the CNN model
# model = Sequential([
#     Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 1), kernel_regularizer=l2(0.01)),
#     MaxPooling2D((2, 2)),
#     Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(0.01)),
#     MaxPooling2D((2, 2)),
#     Conv2D(128, (3, 3), activation='relu', kernel_regularizer=l2(0.01)),
#     MaxPooling2D((2, 2)),
#     Flatten(),
#     Dense(256, activation='relu', kernel_regularizer=l2(0.01)),
#     Dropout(0.5),
#     Dense(1, activation='sigmoid', kernel_regularizer=l2(0.01))  # 2 classes: pneumonia, normal
# ])

# model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 1), kernel_regularizer=l2(0.005)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(0.005)),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu', kernel_regularizer=l2(0.005)),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(256, activation='relu', kernel_regularizer=l2(0.005)),
    Dropout(0.5),
    Dense(1, activation='sigmoid', kernel_regularizer=l2(0.005))  # 2 classes: pneumonia, normal
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

C:\Users\jnpre\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [24]:
# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3)

In [25]:
# Train the model
history = model.fit(train_generator, 
                    validation_data=(X_val, y_val), 
                    epochs=20, 
                    batch_size=batch_size, 
                    )

Epoch 1/20


C:\Users\jnpre\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


163/163 ━━━━━━━━━━━━━━━━━━━━ 190s 1s/step - accuracy: 0.7293 - loss: 2.3320 - val_accuracy: 0.5294 - val_loss: 1.2278
Epoch 2/20
163/163 ━━━━━━━━━━━━━━━━━━━━ 197s 1s/step - accuracy: 0.7914 - loss: 0.9535 - val_accuracy: 0.9412 - val_loss: 0.8037
Epoch 3/20
163/163 ━━━━━━━━━━━━━━━━━━━━ 197s 1s/step - accuracy: 0.8391 - loss: 0.7329 - val_accuracy: 0.7647 - val_loss: 0.7592
Epoch 4/20
163/163 ━━━━━━━━━━━━━━━━━━━━ 184s 1s/step - accuracy: 0.8381 - loss: 0.6412 - val_accuracy: 0.6471 - val_loss: 0.8809
Epoch 5/20
163/163 ━━━━━━━━━━━━━━━━━━━━ 184s 1s/step - accuracy: 0.8621 - loss: 0.5763 - val_accuracy: 0.6471 - val_loss: 0.8345
Epoch 6/20
163/163 ━━━━━━━━━━━━━━━━━━━━ 183s 1s/step - accuracy: 0.8623 - loss: 0.5623 - val_accuracy: 0.7647 - val_loss: 0.6720
Epoch 7/20
163/163 ━━━━━━━━━━━━━━━━━━━━ 185s 1s/step - accuracy: 0.8808 - loss: 0.5164 - val_accuracy: 0.7647 - val_loss: 0.6645
Epoch 8/20
163/163 ━━━━━━━━━━━━━━━━━━━━ 184s 1s/step - accuracy: 0.8789 - loss: 0.4944 - val_accuracy: 0.647

In [26]:
# with l2 .005 and learning_rate .0001
# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_acc}')

20/20 ━━━━━━━━━━━━━━━━━━━━ 4s 174ms/step - accuracy: 0.5465 - loss: 1.1123
Test accuracy: 0.7307692170143127


In [28]:
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype("int32")
confusion_matrix(y_test, y_pred)

20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 151ms/step


array([[ 85, 149],
       [ 19, 371]], dtype=int64)

In [21]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 1), kernel_regularizer=l2(0.005)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(0.005)),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu', kernel_regularizer=l2(0.005)),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(256, activation='relu', kernel_regularizer=l2(0.005)),
    Dense(1, activation='sigmoid', kernel_regularizer=l2(0.005))  # 2 classes: pneumonia, normal
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

In [22]:
# Train the model
history = model.fit(train_generator, 
                    validation_data=(X_val, y_val), 
                    epochs=20, 
                    batch_size=batch_size, 
                    )

Epoch 1/20
163/163 ━━━━━━━━━━━━━━━━━━━━ 146s 885ms/step - accuracy: 0.7448 - loss: 2.2021 - val_accuracy: 0.7647 - val_loss: 0.9614
Epoch 2/20
163/163 ━━━━━━━━━━━━━━━━━━━━ 140s 856ms/step - accuracy: 0.8243 - loss: 0.8591 - val_accuracy: 0.5294 - val_loss: 1.4617
Epoch 3/20
163/163 ━━━━━━━━━━━━━━━━━━━━ 161s 982ms/step - accuracy: 0.8489 - loss: 0.6920 - val_accuracy: 0.6471 - val_loss: 1.0063
Epoch 4/20
163/163 ━━━━━━━━━━━━━━━━━━━━ 159s 969ms/step - accuracy: 0.8690 - loss: 0.6023 - val_accuracy: 0.6471 - val_loss: 0.9380
Epoch 5/20
163/163 ━━━━━━━━━━━━━━━━━━━━ 160s 978ms/step - accuracy: 0.8695 - loss: 0.5704 - val_accuracy: 0.7647 - val_loss: 0.6563
Epoch 6/20
163/163 ━━━━━━━━━━━━━━━━━━━━ 157s 960ms/step - accuracy: 0.8700 - loss: 0.5302 - val_accuracy: 0.7647 - val_loss: 0.6584
Epoch 7/20
163/163 ━━━━━━━━━━━━━━━━━━━━ 158s 964ms/step - accuracy: 0.8701 - loss: 0.5361 - val_accuracy: 0.7647 - val_loss: 0.6053
Epoch 8/20
163/163 ━━━━━━━━━━━━━━━━━━━━ 158s 963ms/step - accuracy: 0.8810 -

In [24]:
# with l2 .005 and learning_rate .0001 and removed dropdout
# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_acc}')

20/20 ━━━━━━━━━━━━━━━━━━━━ 4s 180ms/step - accuracy: 0.5628 - loss: 1.0626
Test accuracy: 0.7339743375778198
